# Jupyter Notebook: Parsing CSV and Visualizing Data

In this notebook, we will load a CSV file, convert it to a pandas DataFrame, and then create map visualizations using Folium.

In [ ]:
import piplite
await piplite.install(['pandas'])
await piplite.install(['geopandas'])
await piplite.install(['folium'])

In [ ]:
# Import necessary libraries
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster

## Step 1: Load CSV File and Parse Data

In [ ]:
# Load NUFORC data (through 2013)
nuforc_data = pd.read_csv('scrubbed.csv', low_memory=False)
# Check column names
print(nuforc_data.columns)

## Step 2: Check for Invalid Values in Latitude and Longitude

In [ ]:
# Check for non-numeric values in 'latitude' and 'longitude'
invalid_lat = nuforc_data[pd.to_numeric(nuforc_data['latitude'], errors='coerce').isna()]
invalid_lon = nuforc_data[pd.to_numeric(nuforc_data['longitude'], errors='coerce').isna()]

# Print invalid latitude and longitude entries
print("Invalid Latitude Entries:")
print(invalid_lat)

print("\nInvalid Longitude Entries:")
print(invalid_lon)

## Step 3: Remove or Correct Invalid Entries

In [ ]:
# Convert 'latitude' and 'longitude' columns to numeric, setting invalid entries to NaN
nuforc_data['latitude'] = pd.to_numeric(nuforc_data['latitude'], errors='coerce')
nuforc_data['longitude'] = pd.to_numeric(nuforc_data['longitude'], errors='coerce')

# Drop rows with NaN values in 'latitude' or 'longitude'
nuforc_data = nuforc_data.dropna(subset=['latitude', 'longitude'])

## Step 4: Create the Searchable Global Map Using Folium

In [ ]:
# Initialize a map centered globally (e.g., on the USA)
ufo_map = folium.Map(location=[0, 0], zoom_start=2)

# Create a marker cluster for better visualization
marker_cluster = MarkerCluster().add_to(ufo_map)

# Add sightings to the map
for idx, sighting in nuforc_data.iterrows():
    # Ensure latitude and longitude data are available
    if not pd.isna(sighting['latitude']) and not pd.isna(sighting['longitude']):
        # Create a popup with sighting details
        popup_text = f"Sighting Date: {sighting['datetime']}<br>City: {sighting['city']}<br>State: {sighting['state']}<br>Country: {sighting['country']}<br>Shape: {sighting['shape']}<br>Duration: {sighting['duration (hours/min)']}<br>Description: {sighting['comments']}"
        
        # Add each sighting to the map
        folium.Marker(
            location=[sighting['latitude'], sighting['longitude']],
            popup=folium.Popup(popup_text, max_width=300)
        ).add_to(marker_cluster)

# Show the map
ufo_map

## Step 5: Make the Map Searchable

In [ ]:
from folium.plugins import Search

# Add a search feature based on the marker's 'popup' property (in this case, UFO descriptions)
search = Search(
    layer=marker_cluster,
    geom_type='Point',
    placeholder='Search for a sighting...',
    collapsed=False,
).add_to(ufo_map)

# Display the map with search enabled
ufo_map

## Step 6: Export or Save the Map

In [ ]:
# Save the map as an HTML file
ufo_map.save('ufo_sightings_map.html')